#GPT3.5

In [78]:
pip install openai


In [79]:
pip install nltk

In [84]:
# prompt: load from google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
import pandas as pd
# Load the CSV file

path = '/content/drive/MyDrive//pcems/strat_test_set.csv'
df_test = pd.read_csv(path)
df_test.head()

,sentence,exercise_segment,feeling_segment,intensity,duration
0,I hiked 6 miles for 1.5 hrs. I felt really...,Hiking,sore,medium,1.5 hrs
1,I shot the basketball for an hour I felt happy,Basketball,happy,medium,1 hr
2,I ran for 45 minutes on the treadmill . I felt...,Running,"tired , out of shape",high,45 mins
3,I ran for 30 minutes I felt tired in the begin...,Running,energized,high,30 mins
4,I squatted for 20 minutes I was exhausted afte...,Squats,exhausted,medium,20 mins


In [96]:
df_test.shape

(24, 5)

In [101]:
import openai
import pandas as pd
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from transformers import BertTokenizer, BertModel
import torch

# Initialize the OpenAI client with your API key
openai.api_key = ''

# Sample training and test DataFrames (replace with your actual data)
df_train = pd.DataFrame({
    'sentence': [
        'I did yoga from a video on tv for 30 mins. I was so sore from all the stretches.',
        'I used my ab roller for 15 minutes straight. I was so fatigued afterwards!',
        'I walked 3 miles in 40 minutes. Wow, I feel excited, that was great.',
        'I did the treadmill and ran for 30 minutes. I felt really tired.',
        'I went swimming for two hours today. I was exhausted afterwards.'
    ],
    'exercise_segment': ['yoga', 'abdominals', 'walk', 'treadmill', 'swim'],
    'feeling_segment': ['sore', 'fatigued', 'great', 'tired', 'exhausted'],
    'intensity': ['low', 'low', 'medium', 'high', 'high'],
    'duration': ['30 mins', '15 mins', '40 mins', '30 mins', '2 hrs']
})


# Create the prompt from the training data, limiting to a few examples
K = 5  # Number of few-shot examples

def get_predictions(log_entry, few_shot_prompt):
    prompt = few_shot_prompt + f"Log: {log_entry}\nExercise Tag:"
    # Pass the API key to the client instance
    client = OpenAI(api_key=openai.api_key)  # Replace with your actual API key
    response = client.completions.create(
        model='gpt-3.5-turbo-instruct',
        prompt=prompt,
        max_tokens=50,
        stop=['\n\n']
    )

    # Extract the predicted Exercise Tag, Feeling Tag, Intensity, and Duration
    prediction = response.choices[0].text.strip()
    lines = prediction.split('\n')

    exercise_tag = lines[0].replace('Exercise Tag: ', '').strip()
    feeling_tag = lines[1].replace('Feeling Tag: ', '').strip()
    intensity = lines[2].replace('Intensity: ', '').strip()
    duration = lines[3].replace('Duration: ', '').strip()
    return exercise_tag, feeling_tag, intensity, duration

def classify_text(log, conditioning_examples):
    few_shot_prompt = ""
    for example in conditioning_examples:
        few_shot_prompt += (
            f"Log: {example['sentence']}\n"
            f"Exercise Tag: {example['exercise_segment']}\n"
            f"Feeling Tag: {example['feeling_segment']}\n"
            f"Intensity: {example['intensity']}\n"
            f"Duration: {example['duration']}\n\n"
        )
    return get_predictions(log, few_shot_prompt)

# Initialize the stemmer
stemmer = PorterStemmer()

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Helper function to stem words
def stem_word(word):
    return stemmer.stem(word)

# Helper function to compute BERT embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# Prepare true labels and predictions
true_exercise_tags = []
true_feeling_tags = []
true_intensities = []
true_durations = []

predicted_exercise_tags = []
predicted_feeling_tags = []
predicted_intensities = []
predicted_durations = []

for _, row in df_test.iterrows():
    new_log = row['sentence']
    conditioning_examples = random.sample(df_train.to_dict('records'), min(K, len(df_train)))
    exercise_tag, feeling_tag, intensity, duration = classify_text(new_log, conditioning_examples)

    true_exercise_tags.append(stem_word(row['exercise_segment']))
    true_feeling_tags.append(stem_word(row['feeling_segment']))
    true_intensities.append(row['intensity'])
    true_durations.append(row['duration'])

    predicted_exercise_tags.append(stem_word(exercise_tag))
    predicted_feeling_tags.append(stem_word(feeling_tag))
    predicted_intensities.append(intensity)
    predicted_durations.append(duration)

# Store true and predicted values in a dataframe
df_test['true_exercise_tags'] = true_exercise_tags
df_test['predicted_exercise_tags'] = predicted_exercise_tags
df_test['true_feeling_tags'] = true_feeling_tags
df_test['predicted_feeling_tags'] = predicted_feeling_tags
df_test['true_intensities'] = true_intensities
df_test['predicted_intensities'] = predicted_intensities
df_test['true_durations'] = true_durations
df_test['predicted_durations'] = predicted_durations

# Calculate semantic similarity
def calculate_semantic_similarity(true_list, predicted_list):
    similarities = []
    for true_text, pred_text in zip(true_list, predicted_list):
        true_embedding = get_embedding(true_text)
        pred_embedding = get_embedding(pred_text)
        similarity = cosine_similarity([true_embedding], [pred_embedding])[0][0]
        similarities.append(similarity)
    return similarities

exercise_tag_similarities = calculate_semantic_similarity(true_exercise_tags, predicted_exercise_tags)
feeling_tag_similarities = calculate_semantic_similarity(true_feeling_tags, predicted_feeling_tags)
intensity_tag_similarities = calculate_semantic_similarity(true_intensities, predicted_intensities)


# Apply semantic similarity threshold for accuracy
def calculate_custom_accuracy(true_tags, predicted_tags, similarities, threshold=0.7):
    correct_matches = sum(1 for true_tag, pred_tag, sim in zip(true_tags, predicted_tags, similarities) if true_tag == pred_tag or sim >= threshold)
    return correct_matches / len(true_tags)

exercise_tag_accuracy = calculate_custom_accuracy(true_exercise_tags, predicted_exercise_tags, exercise_tag_similarities)
feeling_tag_accuracy = calculate_custom_accuracy(true_feeling_tags, predicted_feeling_tags, feeling_tag_similarities)
intensity_accuracy = calculate_custom_accuracy(true_intensities, predicted_intensities, intensity_tag_similarities)
#intensity_accuracy = accuracy_score(true_intensities, predicted_intensities)
duration_accuracy = accuracy_score(true_durations, predicted_durations)

print(f"Exercise Tag Accuracy: {exercise_tag_accuracy:.4f}")
print(f"Feeling Tag Accuracy: {feeling_tag_accuracy:.4f}")
print(f"Intensity Accuracy: {intensity_accuracy:.4f}")
print(f"Duration Accuracy: {duration_accuracy:.4f}")



Exercise Tag Accuracy: 0.8750
Feeling Tag Accuracy: 0.8750
Intensity Accuracy: 0.9583
Duration Accuracy: 1.0000


In [100]:

 df_test[['sentence', 'predicted_intensities','true_intensities']]

,sentence,predicted_intensities,true_intensities
0,I hiked 6 miles for 1.5 hrs. I felt really...,medium-high,medium
1,I shot the basketball for an hour I felt happy,medium,medium
2,I ran for 45 minutes on the treadmill . I felt...,high,high
3,I ran for 30 minutes I felt tired in the begin...,high,high
4,I squatted for 20 minutes I was exhausted afte...,medium,medium
5,I did yoga for 1 hour today . I feel energized !,low,medium
6,i climb 700 steps on the stair climber for 40 ...,high,high
7,I biked one hour My legs hurt a lot !,high,high
8,i just finish 20 push ups in 10 mins and am w...,high,medium
9,I ran 20 minutes and I was feeling great .,medium,medium


In [91]:
df_test

,sentence,exercise_segment,feeling_segment,intensity,duration,true_exercise_tags,predicted_exercise_tags,true_feeling_tags,predicted_feeling_tags,true_intensities,predicted_intensities,true_durations,predicted_durations
0,I hiked 6 miles for 1.5 hrs. I felt really...,Hiking,sore,medium,1.5 hrs,hike,hike,sore,sore,medium,medium,1.5 hrs,1.5 hrs
1,I shot the basketball for an hour I felt happy,Basketball,happy,medium,1 hr,basketbal,basketbal,happi,happi,medium,medium,1 hr,1 hr
2,I ran for 45 minutes on the treadmill . I felt...,Running,"tired , out of shape",high,45 mins,run,treadmil,"tired , out of shap","tired, out of shap",high,high,45 mins,45 mins
3,I ran for 30 minutes I felt tired in the begin...,Running,energized,high,30 mins,run,run,energ,energ,high,high,30 mins,30 mins
4,I squatted for 20 minutes I was exhausted afte...,Squats,exhausted,medium,20 mins,squat,squat,exhaust,exhaust,medium,medium,20 mins,20 mins
5,I did yoga for 1 hour today . I feel energized !,Yoga,energized,low,1 hr,yoga,yoga,energ,energ,low,medium,1 hr,1 hr
6,i climb 700 steps on the stair climber for 40 ...,Stair Climbing,exhausted,high,40 mins,stair climb,stair climb,exhaust,exhaust,high,high,40 mins,40 mins
7,I biked one hour My legs hurt a lot !,bike,sore,high,1 hr,bike,bike,sore,sore,high,medium,1 hr,1 hr
8,i just finish 20 push ups in 10 mins and am w...,Chest press,worn out,high,10 mins,chest press,push up,worn out,worn out,high,medium,10 mins,10 mins
9,I ran 20 minutes and I was feeling great .,Running,great,medium,20 mins,run,run,great,great,medium,medium,20 mins,20 mins


In [88]:
# prompt: insert actual and predicted values  in new dataframe

import pandas as pd
# Create a new DataFrame to store actual and predicted values
df_results = pd.DataFrame({
    'Sentence': df_test['sentence'],
    'Actual Exercise Tag': df_test['exercise_segment'],
    'Predicted Exercise Tag': df_test['predicted_exercise_tags'],
    'Actual Feeling Tag': df_test['feeling_segment'],
    'Predicted Feeling Tag': df_test['predicted_feeling_tags'],
    'Actual Intensity': df_test['intensity'],
    'Predicted Intensity': df_test['predicted_intensities'],
    'Actual Duration': df_test['duration'],
    'Predicted Duration': df_test['predicted_durations']
})

df_results


KeyError: 'predicted_exercise_tags'